In [1]:
import pandas as pd
import re
import torch
import json
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from sklearn.model_selection import train_test_split

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f083b512590>>
Traceback (most recent call last):
  File "/home/vlad/bralex/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [2]:
df = pd.read_csv('../labeled_and_validated_10k.csv')
df = df[df.Label_gpt == 'valid']
df = df[['Text', 'Title', 'Price', 'Currency', 'Count']].reset_index(drop=True)
df['json'] = pd.Series([[{'Title': row.Title, 'Price': row.Price, 'Currency': row.Currency, 'Count': row.Count}] for row in df.itertuples()])
text_df = df[['Text', 'json']].groupby('Text').sum().reset_index()
text_df['n_bundle'] = text_df.json.apply(len)
text_df.head()

,Text,json,n_bundle
0,! новая. хлопковая рубашка оверсайз с приятной...,"[{'Title': 'хлопковая рубашка оверсайз', 'Pric...",1
1,!!! ПОМИДОРКИ !!! в 3л Банке.... по 250 руб. К...,"[{'Title': 'Икра КРАСНАЯ 140 гр. банка', 'Pric...",3
2,"""конфетный мячик"" 🍬🏀 цена €7😻 бесплатная доста...","[{'Title': 'конфетный мячик', 'Price': '7', 'C...",1
3,# ТРИКО Распродажа без выбораМОДЕЛЬ ТКАНЬ ТРИК...,"[{'Title': 'Трико, двух нитки качества супер',...",1
4,# ТРИКО Распродажа без выбораМОДЕЛЬ ТКАНЬ ТРИК...,[{'Title': 'Упаковка двуниточного трикотажа ра...,1


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruT5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/ruT5-large")

In [5]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# device = 'cuda'
# model_path = 'ruT5-large-too-good'
# model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
# tokenizer = AutoTokenizer.from_pretrained(model_path)

In [17]:
tokenizer.add_tokens(["<BOB>", "<EOB>", "<BOT>", "<EOT>", "<BOP>", "<EOP>", "<BOC1>", "<EOC1>", "<BOC2>", "<EOC2>"])
model.resize_token_embeddings(len(tokenizer))

def process_json(json):
  return ''.join([f"<BOB><BOT>{d['Title']}<EOT><BOP>{d['Price']}<EOP><BOC1>{d['Count']}<EOC1><BOC2>{d['Currency']}<EOC2><EOB>" for d in json])

def unprocess_json(s):
  json = []
  for t in re.findall(r'<BOB>(.*?)<EOB>', s):
    try:
      json.append({
        'Title': re.findall(r'<BOT>(.*?)<EOT>', t)[0],
        'Price': re.findall(r'<BOP>(.*?)<EOP>', t)[0],
        'Count': re.findall(r'<BOC1>(.*?)<EOC1>', t)[0],
        'Currency': re.findall(r'<BOC2>(.*?)<EOC2>', t)[0]
      })
    except Exception as e:
      print(t)
      raise e
  return json
assert text_df.loc[0, 'json'] == unprocess_json(process_json(text_df.loc[0, 'json']))

In [5]:
# input_text = """Продам беговую дорожку электрическую, пользовались мало. Состояние очень хорошее, в основном стояла без использования! Цена 16.000₽"""
# input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# outputs = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)
# decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
# unprocess_json(decoded_output)

In [59]:
from datasets import Dataset

ads_dataset = Dataset.from_pandas(text_df[["Text", "json"]])
ads_dataset = ads_dataset.train_test_split(test_size=0.2, seed=42)
# ads_dataset = ads_dataset.train_test_split(test_size=0.2)
ads_dataset = ads_dataset.flatten()
ads_dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'json'],
        num_rows: 5803
    })
    test: Dataset({
        features: ['Text', 'json'],
        num_rows: 1451
    })
})

In [60]:
def preprocess_function(examples):
    inputs = [text for text in examples["Text"]]
    # targets = ['' for bundles in examples["json"]]
    targets = [process_json(bundles) for bundles in examples["json"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

ads = ads_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=ads_dataset["train"].column_names
)
ads = ads.flatten()

Map (num_proc=4):   0%|          | 0/5803 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1451 [00:00<?, ? examples/s]

In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [9]:
from transformers import DataCollatorForSeq2Seq

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

In [10]:
def compute_metrics(eval_pred):
  preds, labels = eval_pred
  preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  print(preds)
  return {'preds':preds}

In [11]:
n_epochs = 6

training_args = Seq2SeqTrainingArguments(
    output_dir="ruT5-large",
    # overwrite_output_dir=True,
    evaluation_strategy="epoch",
    # eval_steps=1,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=n_epochs,
    predict_with_generate=True,
    generation_max_length=256,
    fp16=True,
    lr_scheduler_type="cosine",
    group_by_length=False,
    warmup_steps=3,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=ads["train"],
    eval_dataset=ads["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.441217
2,0.896100,0.406197
3,0.404100,0.392848
4,0.404100,0.385914
5,0.302900,0.388180
6,0.255500,0.390847


TrainOutput(global_step=2178, training_loss=0.4465430849073566, metrics={'train_runtime': 1225.1362, 'train_samples_per_second': 28.42, 'train_steps_per_second': 1.778, 'total_flos': 1.825158127872e+16, 'train_loss': 0.4465430849073566, 'epoch': 6.0})

In [28]:
device = 'cuda'

def generate_text(input_ids):
  model.eval()
  input_ids = torch.IntTensor([input_ids['input_ids']]).to(device)
  with torch.no_grad():
    generated_ids = model.generate(input_ids, max_length=512)
  return [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

results = []
for batch in tqdm(ads['test']):
  outputs = generate_text(batch)
  results.extend(outputs)

  0%|          | 0/1451 [00:00<?, ?it/s]

In [29]:
results

['<BOB><BOT> Канбан Электро + набор улучшений + запасные компоненты + все карты в протекторах<EOT><BOP> 11500<EOP><BOC1> 1<EOC1><BOC2> RUB<EOC2><EOB>',
 '<BOB><BOT> набор для картины-мозаики своими-руками<EOT><BOP> 7<EOP><BOC1> 1<EOC1><BOC2> EUR<EOC2><EOB>',
 '<BOB><BOT> стул (офисное кресло) икеа<EOT><BOP> 70<EOP><BOC1> 1<EOC1><BOC2> EUR<EOC2><EOB>',
 '<BOB><BOT> сушилка для белья зонтичного типа, складная<EOT><BOP> 20<EOP><BOC1> 1<EOC1><BOC2> EUR<EOC2><EOB>',
 '<BOB><BOT> Подушка бамбук ткань тик размер 70х70см упаковка<EOT><BOP> 2300<EOP><BOC1> 10<EOC1><BOC2> RUB<EOC2><EOB><BOB><BOT> Подушка бамбук ткань тик размер 50х70см упаковка<EOT><BOP> 2000<EOP><BOC1> 10<EOC1><BOC2> RUB<EOC2><EOB><BOB><BOT> Подушка бамбук ткань тик размер 70х70см упаковка<EOT><BOP> 2300<EOP><BOC1> 10<EOC1><BOC2> RUB<EOC2><EOB><BOB><BOT> Подушка бамбук ткань тик размер 70х70см упаковка<EOT><BOP> 2300<EOP><BOC1> 10<EOC1><BOC2> RUB<EOC2><EOB>',
 '<BOB><BOT> Электросамокат ninebot f40<EOT><BOP> 27000<EOP><BOC1> 1<

In [30]:
predictions = []
refs = []
for i in range(len(results)):
  input_text = tokenizer.decode(ads['test'][i]['input_ids'], skip_special_tokens=True)
  print(f"Input: {input_text}")
  print(f"Predicted Output:")
  output_json = results[i]
  try:
    output_json = unprocess_json(results[i])
  except Exception as e:
    print(f'Cannot parse: {results[i]}')
    print(f'Exception: {e}')
    continue
  print(output_json)
  if len(output_json) == 1:
    predictions.append(output_json[0]['Title'])
    ref_text = tokenizer.decode(ads['test'][i]['labels'], skip_special_tokens=True)
    ref_text = unprocess_json(ref_text)[0]['Title']
    refs.append([ref_text])
  print("\n")

Input: БНИ_ПродамБУ_Елец БНИ_Авито БНИ_Boxberry БНИ_СДЭК БНИ_ushakov8882002 Канбан Электро + набор улучшений + запасные компоненты + все карты в протекторах - 11500 руб С Авито доставкой +2% (+5% с 31 августа)
Predicted Output:
[{'Title': ' Канбан Электро + набор улучшений + запасные компоненты + все карты в протекторах', 'Price': ' 11500', 'Count': ' 1', 'Currency': ' RUB'}]


Input: мозаика 30х30 см. комплект всего для картины-мозаики своими-руками. можно вставить в фоторамку. 7 евро. лимассол.
Predicted Output:
[{'Title': ' набор для картины-мозаики своими-руками', 'Price': ' 7', 'Count': ' 1', 'Currency': ' EUR'}]


Input: продам стул( офисное кресло) икеа в идеальном состоянии, использовалось меньше полугода, 70. лимассол
Predicted Output:
[{'Title': ' стул (офисное кресло) икеа', 'Price': ' 70', 'Count': ' 1', 'Currency': ' EUR'}]


Input: сушилка для белья, зонтичного типа, складная. размер 1700х1700 мм. новая, пользовались один раз.20 евро. писсури. доставкой по кипру не занима

In [31]:
len(predictions), len(results)

(1150, 1451)

In [32]:
import evaluate
metric = evaluate.load("sacrebleu")

metric.compute(predictions=predictions, references=refs)

{'score': 51.043811313113956,
 'counts': [4665, 3176, 2094, 1330],
 'totals': [6719, 5569, 4460, 3477],
 'precisions': [69.42997469861587,
  57.02998743041839,
  46.95067264573991,
  38.25136612021858],
 'bp': 0.9884582525763934,
 'sys_len': 6719,
 'ref_len': 6797}

In [33]:
refs

[[' Канбан Электро, набор улучшений, запасные компоненты, все карты в протекторах'],
 [' мозаика 30х30 см, комплект для картины-мозаики своими руками'],
 [' офисное кресло IKEA'],
 [' сушилка для белья, зонтичного типа, складная'],
 [' Электросамокат ninebot f40, новый, запечатанный'],
 [' столи для кровати или дивана'],
 [' вай-фай роутер с разъёмом для симки'],
 [' двойная цепочка с подвеской-сердечком и жемчужинами (бижутерия)'],
 [' MacBook Air 2020'],
 [' Стол компьютерный'],
 [' капсюльная кофеварка tassimo bosch б/у'],
 [' кастомный шоппер wednesday'],
 [' mi smart antibacterial humidifier - увлажнитель воздуха 4,5л'],
 [' кофеварка chemex, состояние хорошее'],
 [' кофемолка ручная timemore c3, полный комплект'],
 [' металлический студийный мольберт fome 2041'],
 [' Электросамокат Ninebot E22'],
 [' Набор из 2 новых сатиновых наволочек темно-синего цвета на молнии'],
 [' MacBook Air (13-inch, Early 2015) 1.6 GHz 2-ядерный процессор Intel Core i5, 8GB RAM'],
 [' винтажная фруктов

In [34]:
predictions

[' Канбан Электро + набор улучшений + запасные компоненты + все карты в протекторах',
 ' набор для картины-мозаики своими-руками',
 ' стул (офисное кресло) икеа',
 ' сушилка для белья зонтичного типа, складная',
 ' Электросамокат ninebot f40',
 ' столи для кровати или дивана',
 ' вай-фай роутер с разъёмом для симки',
 ' двойная цепочка с подвеской-сердечком и жемчужинами',
 ' macbook air 2020 года',
 ' Стол компьютерный',
 ' капсюльная кофеварка tassimo bosch, б/у',
 ' кастомный шоппер wednesday',
 ' mi smart antibacterial humidifier - увлажнитель воздуха 4,5л',
 ' кофеварка chemex с фильтрами Half Moon',
 ' кофемолка ручная timemore c3, полный комплект',
 ' металлический студийный мольберт fome 2041',
 ' Электросамокат Ninebot E22',
 ' набор синих сатиновых наволочек для подушек на молниях',
 ' MacBook Air (13-inch, Early 2015) 1,6 GHz, 2-ядерный процессор, Intel Core i5, 8GB RAM',
 ' фруктовницы винтажные с ручкой',
 ' wanbo t6 max projector white, глобальная версия',
 ' iphone 14 pr

In [63]:
output_dir = "ruT5-large-too-good-try-3"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('ruT5-large-too-good-try-3/tokenizer_config.json',
 'ruT5-large-too-good-try-3/special_tokens_map.json',
 'ruT5-large-too-good-try-3/spiece.model',
 'ruT5-large-too-good-try-3/added_tokens.json',
 'ruT5-large-too-good-try-3/tokenizer.json')